This program attempst predict the mutation score of a function

In [8]:
#Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [20]:
#Collect and clean the data
df = pd.read_json("../new_output_with_functions.json")
list(df.columns.values)

['function_name', 'function_scope', 'patterns', 'mutants', 'mutation_score']

In [21]:
#View data
df

,function_name,function_scope,patterns,mutants,mutation_score
0,parse_mutant_ids,4-6,"[{'lineno': 4, 'coloffset': 0, 'linematch': 'd...",[],NaN
1,get_function_name_from_diff,8-15,"[{'lineno': 8, 'coloffset': 0, 'linematch': 'd...","[{'name': 'Mutant #120', 'line': 11, 'descript...",100.000000
2,main,17-51,"[{'lineno': 17, 'coloffset': 0, 'linematch': '...","[{'name': 'Mutant #123', 'line': 17, 'descript...",95.833333
3,test_hparams,6-33,"[{'lineno': 6, 'coloffset': 0, 'linematch': 'd...","[{'name': 'Mutant #118', 'line': 7, 'descripti...",94.117647
4,test_picable_function,35-51,"[{'lineno': 35, 'coloffset': 0, 'linematch': '...","[{'name': 'Mutant #135', 'line': 36, 'descript...",100.000000
5,test_files,53-74,"[{'lineno': 53, 'coloffset': 0, 'linematch': '...","[{'name': 'Mutant #147', 'line': 53, 'descript...",77.777778
6,test_directories,76-105,"[{'lineno': 76, 'coloffset': 0, 'linematch': '...",[],NaN
7,test_cached,5-73,"[{'lineno': 5, 'coloffset': 0, 'linematch': 'd...","[{'name': 'Mutant #118', 'line': 7, 'descripti...",89.361702
8,__init__,9-11,"[{'lineno': 9, 'coloffset': 4, 'linematch': 'd...","[{'name': 'Mutant #120', 'line': 11, 'descript...",100.000000
9,add_directories,13-27,"[{'lineno': 13, 'coloffset': 4, 'linematch': '...","[{'name': 'Mutant #122', 'line': 13, 'descript...",87.500000


In [18]:
df.to_csv('my_dataframe.csv', index=False)

In [19]:
#Show the data visually

df.plot(x="check_id" , y= "mutation_score")
plt.xticks(rotation=45)

KeyError: 'check_id'

In [ ]:

#Split data into training and testing sets,(80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(df, df['mutation_score'], test_size= 0.2, random_state = 42)

In [5]:
from sklearn.preprocessing import LabelEncoder
#Preprocess data
#Convert text labels "check_id" values into numerical representations that the model can understand and use for learning patterns
def preprocess_data(data):
  le = LabelEncoder() #Create LabelEncoder object
  X_encoded = le.fit_transform(data['pattern']) #map the category with numerical labels

  #a case when we need to add more feature, set None for now
  X_other_features = None

  #Combine features
  if X_other_features is not None:
    X = np.hstack((X_encoded.reshape(-1,1), X_other_features))
  else:
    X=X_encoded.reshape(-1,1) #reshape to ensure correct format in all rows and single column
  return X


In [ ]:
#Preprocess training and testing data
X_train_processed = preprocess_data(X_train.copy())
X_test_processed = preprocess_data(X_test.copy())

In [ ]:
#Define thresholds to categorize mutation score
low_threshold = 25
medium_threshold = 75

#Map mutation score for categories
def map_to_category(score):
  if score is np.NaN:
    return "NA"
  elif score < low_threshold:
    return "low"
  elif score < medium_threshold:
    return "medium"
  else:
    return "high"

In [ ]:
#Convert mutation scores to categories for training
y_train_categories = [map_to_category(score) for score in y_train]
print(y_train_categories)

In [25]:
#Test data with RandomForestClassifier() since we want to classify mutation in three score categories "low", "medium", "high"

from sklearn.ensemble import RandomForestClassifier

In [26]:
model = RandomForestClassifier()
model.fit(X_train_processed,y_train_categories)


RandomForestClassifier()

In [27]:
#Predict catagories on test set
y_pred_categories = model.predict(X_test_processed)
print(y_pred_categories)

['high' 'high' 'high' 'high' 'high' 'high' 'high' 'high' 'high' 'high'
 'high' 'high' 'high' 'high' 'high' 'high' 'high' 'high' 'high' 'high'
 'high' 'high' 'high' 'high' 'high' 'high' 'high' 'high' 'high' 'high'
 'high' 'high' 'high' 'high' 'high']


In [28]:
#Map predicted categories back to low_medium_high labels
y_pred_labels = [map_to_category(score) for score in model.predict(X_test_processed)]

TypeError: '<' not supported between instances of 'numpy.str_' and 'int'